
*Technical University of Munich<br>
Professorship of Environmental Sensing and Modeling<br><br>*
**Author:**  Daniel Kühbacher<br>
**Date:**  27.10.2023

--- 

# VISUM Model Preprocessing

<!--Notebook description and usage information-->
This notebook is to process the raw VISUM traffic model export and convert the data into gpkg files.

In [2]:
import sys

import pandas as pd
import geopandas as gpd

# import custom modules
sys.path.append('../../utils/')
import data_paths

In [3]:
# import visum data
visum_link_path = data_paths.VISUM_FOLDER_PATH + "analyse_2019_link.SHP"
visum_zone_path = data_paths.VISUM_FOLDER_PATH + "analyse_2019_zone.SHP"

visum_links = gpd.read_file(visum_link_path)
visum_zones = gpd.read_file(visum_zone_path)

print(f"Coordinate reference system of VISUM link data: {visum_links.crs}")
print(f"Coordinate reference system of VISUM zones data: {visum_zones.crs}")

Coordinate reference system of VISUM link data: EPSG:25832
Coordinate reference system of VISUM zones data: EPSG:25832


In [4]:
def hbefa_road_type(visum_typeno) -> str: 
    """returns hbefa compatible road type for all road types in the visum traffic model of Munich.
    The information below was retrieved from the official documentation of the VIUSM model and 
    matched to HBEFA types by intuition and exploration.
    
    0-10: gesperrte und fiktive Strecken sowie Fuß- und Radwege
    100-199: Autobahnen
    200-299: Bundesstraßen
    300-399: Staatsstraßen
    400-499: Kreisstraßen
    500-599: Gemeindestraßen
    600-699: Hauptstraßen (innerorts)
    700-799: Erschließungsstraßen
    800-899: Mittlerer Ring
    900-999: ÖV Strecken

    Args:
        visum_typeno (string object): defines the road type in the VISUM model

    Returns:
        str: HBEFA compatible road type
    """
    
    int_typeno = int(visum_typeno)
    
    if 0 <= int_typeno <=10: 
        return 'none'
    if 100 <= int_typeno <=199: 
        return 'motorway'
    if 200 <= int_typeno <=299: 
        return 'primary-national'
    if 300 <= int_typeno <=399: 
        return 'primary-national'
    if 400 <= int_typeno <=499: 
        return 'secondary'
    if 500 <= int_typeno <=599: 
        return 'collector'
    if 600 <= int_typeno <=699: 
        return 'primary-city'
    if 700 <= int_typeno <=799: 
        return 'residential'
    if 800 <= int_typeno <=899: 
        return 'city-motorway'
    if 900 <= int_typeno <=999: 
        return 'none'
    else: 
        return 'none'

In [7]:
# process and save vism links data
visum_links['road_type'] = visum_links.apply(lambda row: hbefa_road_type(row['TYPENO']), axis = 1)
visum_links.to_file(data_paths.VISUM_FOLDER_PATH+'visum_links.gpkg', driver='GPKG')